In [36]:
import pandas as pd
from unidecode import unidecode
import numpy as np

class CleaningText:

    def __init__(self) -> None:
        pass

    def norm_header(self, bd):
        bd.rename(columns=lambda col: unidecode(col), inplace=True)
        bd.rename(columns=lambda col: col.strip('.!? \n\t').lower(), inplace=True)
        bd.rename(columns=lambda col: col.replace('.', '').replace(' ', '_'), inplace = True)
        return bd

    def col_duplicados(bd, col):
        cols = []
        n = 1
        for col in bd.columns:
            if col == 'estado':
                cols.append(f'estado_{n}')
                n += 1
                continue
            cols.append(col)
        bd.columns = cols
        return bd

    def strip_symbols(col):
        return col.str.strip('.!?$ \n\t')
    
    def lower_col(col):
        return col.str.lower()

    def to_number(col ):
        res = col.str.strip('.!?$ \n\t')
        res = res.str.replace('.','', regex=False)
        # Si f no es dígito => nan 
        res = res.fillna('N/A')
        res.loc[~res.str.isdigit()] = np.nan
        # Convertir a num 
        res = pd.to_numeric(res)
        return res 

    def clean_fnum(col):
        res = col.fillna('nan')
        res = res.str.replace(r'([^0-9])', '', regex=True)
        # Si f = 0 => nan 
        res.loc[res=='0'] = np.nan
        # Si f no es dígito => nan 
        res = res.fillna('nan')
        res.loc[~res.str.isdigit()] = np.nan
        return res 
    
    def clean_num(col):
        res = col.str.replace(r'([^0-9,])', '', regex=True) # Conservar la coma para temas de decimales en bd
        # Si f no es dígito => nan 
        res = res.fillna('nan')
        res = res.apply(lambda x: x.split(',')[0]) # TODO temporal - cambiar por método númerico
        res.loc[~res.str.isdigit()] = np.nan
        return res 
    
    def clean_str(col):
        res = col.fillna('nan')
        res = res.apply(unidecode)
        res = res.str.replace(r'([^a-zA-Z0-9-+(). ])', '', regex=True)
        res = res.str.strip()
        res = res.str.lower()
        return res 

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', 500)
from unidecode import unidecode
import numpy as np
from datetime import datetime
date = datetime.now().strftime("%y%m%d")


In [2]:
df_tes = pd.read_csv('output/bases/220804-1155-220804_nomalizaciones_tesoreria-output.csv', sep = ';', dtype = str, usecols= ['SS', 'CC', 'OC', 'F12', 'TIPO DE DEVOLUCION','VALOR DE DEVOLUCION', 'FECHA ENVIO A RED', 'ESTADO', 'FECHA APROBACION DE LA RED', 'COD AUT NC'])

FileNotFoundError: [Errno 2] No such file or directory: 'output/bases/220804-1155-220804_nomalizaciones_tesoreria-output.csv'

In [ ]:
df_tes.shape

(399857, 10)

In [ ]:

df_tes.rename(columns=lambda col: unidecode(col), inplace=True)
df_tes.rename(columns=lambda col: col.strip('.!? \n\t').lower(), inplace=True)
df_tes.rename(columns=lambda col: col.replace('.', '').replace(' ', '_'), inplace = True)


In [ ]:
df_tes['fecha_aprobacion_de_la_red'][40001]

'2022-04-27'

In [ ]:
columns =  ['tipo_de_devolucion', 'estado']
for cols in columns:
    df_tes[cols] = df_tes[cols].str.lower()
    df_tes[cols] = df_tes[cols].str.strip('.!?$ \n\t')

In [ ]:
columns = ['cc', 'oc', 'f12','cod_aut_nc']
for cols in columns:
    df_tes['temp'] = df_tes[cols].str.extract(r'^(\d{0,})')
    df_tes[cols] = df_tes['temp']
    df_tes.drop(columns='temp', inplace=True)

In [ ]:
df_tes['valor_de_devolucion'] = df_tes['valor_de_devolucion'].str.strip('.!?$ \n\t')
df_tes['valor_de_devolucion'] = df_tes['valor_de_devolucion'].str.replace('.','', regex=False)
# Si f no es dígito => nan 
df_tes['valor_de_devolucion'] = df_tes['valor_de_devolucion'].fillna('N/A')
df_tes['valor_de_devolucion'].loc[~df_tes['valor_de_devolucion'].str.isdigit()] = np.nan
# Convertir a num 


In [ ]:
df_tes['valor_de_devolucion'] = pd.to_numeric(df_tes['valor_de_devolucion']) 
df_tes['valor_de_devolucion'] 

0         239960.0
1         207480.0
2         163794.0
3         236890.0
4         229990.0
            ...   
399852    294000.0
399853    253380.0
399854    709680.0
399855    709680.0
399856    253380.0
Name: valor_de_devolucion, Length: 399857, dtype: float64

In [ ]:
df_tes['fecha_envio_a_red'] = pd.to_datetime(df_tes['fecha_envio_a_red'], format= '%Y-%m-%d')

In [ ]:
df_tes['fecha_aprobacion_de_la_red'] = pd.to_datetime(df_tes['fecha_aprobacion_de_la_red'])

In [ ]:
df_tes['fecha_aprobacion_de_la_red'][4050]

Timestamp('2022-07-26 00:00:00')

In [ ]:
# df_tes.to_csv(f'output/bases/{date}_tesoreria.scv', sep=',')

In [ ]:
df_tes.loc[df_tes.duplicated('cod_aut_nc', keep=False)  & (df_tes['cod_aut_nc'] != "0")]

,ss,cc,oc,f12,tipo_de_devolucion,valor_de_devolucion,fecha_envio_a_red,estado,fecha_aprobacion_de_la_red,cod_aut_nc
6,1-130391060662,1053830729,0,0,parcial,79990,2022-08-03,en proceso enviada a red,NaT,1080096068
14,1-130380978188,1061723629,0,0,total,7647900,2022-08-03,en proceso enviada a red,NaT,1080085741
21,1-130380033008,38247477,0,0,total,3000000,2022-08-03,en proceso enviada a red,NaT,1080086205
22,1-130380033008,38247477,0,0,total,3000900,2022-08-03,en proceso enviada a red,NaT,1080086205
24,1-130391643507,13364239,0,12695695904,parcial,2399901,2022-08-03,en proceso enviada a red,NaT,1080090103
...,...,...,...,...,...,...,...,...,...,...
260162,1-91447692334,761049,1373727554,12577048750,parcial,119644,2020-09-22,procesada,2020-09-25,1060547139
260169,1-91528521445,43543229,1414271806,12603613341,parcial,310360,2020-09-22,procesada,2020-09-25,1060282481
264309,1-91021689424,79653111,1411913239,12601824534,total,43690,2020-09-21,rechazada,2020-09-23,1060137798
264328,1-91021689424,79653111,1411913239,12601824534,total,43690,2020-09-21,rechazada,2020-09-23,1060137798


In [ ]:
df_tes.loc[df_tes["cod_aut_nc"] == '1080096068' ]

,ss,cc,oc,f12,tipo_de_devolucion,valor_de_devolucion,fecha_envio_a_red,estado,fecha_aprobacion_de_la_red,cod_aut_nc
6,1-130391060662,1053830729,0,0,parcial,79990,2022-08-03,en proceso enviada a red,NaT,1080096068
154,1-129917521639,1053830729,1550536674,12694331629,parcial,79990,2022-08-03,en proceso enviada a red,NaT,1080096068


In [ ]:
df_tes.loc[df_tes["cod_aut_nc"] == '1060775471' ]

,ss,cc,oc,f12,tipo_de_devolucion,valor_de_devolucion,fecha_envio_a_red,estado,fecha_aprobacion_de_la_red,cod_aut_nc
247986,1-90185938519,1017217366,1381291319,12582520111,parcial,83770,2020-10-02,rechazada,2020-10-05,1060775471
268351,1-90185938519,1017217366,1381291319,12582520111,total,128765,2020-06-26,rechazada,2020-09-23,1060775471


In [ ]:
df_tes.loc[df_tes["cod_aut_nc"] == '1073558313' ]

,ss,cc,oc,f12,tipo_de_devolucion,valor_de_devolucion,fecha_envio_a_red,estado,fecha_aprobacion_de_la_red,cod_aut_nc
75179,1-118805472700,1193419133,1512195217,12667694416,total,355956.0,2022-01-05,procesada,2022-01-06,1073558313
75180,1-118805472700,1193419133,1512195217,12667694416,total,355956.0,2022-01-05,procesada,2022-01-06,1073558313


In [ ]:
df_tes['fecha_envio_a_red'].min()

Timestamp('2019-12-11 00:00:00')

In [ ]:
df_tes.loc[(df_tes['fecha_envio_a_red'] >= '2022-01-01') & ((df_tes['estado'] == 'procesada'))]['valor_de_devolucion'].sum()

29901949009.0

In [ ]:
df_21_pro = df_tes.loc[(df_tes['cod_aut_nc'] != "0") & (df_tes['fecha_envio_a_red'] >= '2022-01-01') & ((df_tes['estado'] == 'procesada'))]

In [ ]:
df_21_pro.loc[df_21_pro.duplicated('cod_aut_nc', keep=False)]

,ss,cc,oc,f12,tipo_de_devolucion,valor_de_devolucion,fecha_envio_a_red,estado,fecha_aprobacion_de_la_red,cod_aut_nc
997,1-130043579707,53072862,0,12698447333,total,8953900.0,2022-08-01,procesada,2022-08-02,1080028861
998,1-129950884546,51607816,0,12698216083,total,7249900.0,2022-08-01,procesada,2022-08-02,1080037115
1002,1-130265288000,86075961,0,0,total,8953900.0,2022-08-01,procesada,2022-08-02,1080028861
1005,1-130278150072,51607816,0,0,total,7249900.0,2022-08-01,procesada,2022-08-02,1080037115
2230,1-128695318208,63318827,1553592051,12696366179,total,19900.0,2022-07-28,procesada,2022-07-28,
...,...,...,...,...,...,...,...,...,...,...
75179,1-118805472700,1193419133,1512195217,12667694416,total,355956.0,2022-01-05,procesada,2022-01-06,1073558313
75180,1-118805472700,1193419133,1512195217,12667694416,total,355956.0,2022-01-05,procesada,2022-01-06,1073558313
75199,1-118006348301,51943748,0,0,total,92691.0,2022-01-05,procesada,2022-01-06,
75341,1-118796506547,1133348,1514122695,12669050470,parcial,100000.0,2022-01-04,procesada,2022-01-05,
